# OpenAPI APIをBedrock AgentCore Gatewayを使ってMCPツールに変換する

## 概要
Bedrock AgentCore Gatewayは、お客様が既存のAPIを完全に管理されたMCPサーバーに変換できるようにする方法を提供します。インフラストラクチャやホスティングを管理する必要はありません。お客様はOpenAPI仕様をJSONまたはYAMLで持ち込むことができます。OAuth2で保護されたエンタープライズサポートAPIを使用する顧客サービスエージェントを実演します。

Gatewayのワークフローには、エージェントを外部ツールに接続するための以下のステップが含まれます:
* **Gatewayのツールを作成する** - OpenAPI仕様などのスキーマを使用してツールを定義します。OpenAPI仕様は、Amazon Bedrock AgentCoreによってパースされ、Gatewayが作成されます。
* **Gatewayエンドポイントを作成する** - 受信認証付きのMCPエントリポイントとしてゲートウェイを作成します。
* **Gatewayにターゲットを追加する** - ゲートウェイがリクエストを特定のツールにルーティングする方法を定義するOpenAPIターゲットを構成します。OpenAPIファイルの一部のすべてのAPIはMCP互換のツールになり、Gatewayエンドポイント URL を介して利用可能になります。各OpenAPI Gatewayターゲットに対して、OAuthを使用した送信認証を構成します。
* **エージェントコードを更新する** - エージェントをGatewayエンドポイントに接続して、統一されたMCPインターフェースを介してすべての構成済みツールにアクセスします。

![動作の仕組み](images/openapis-oauth-gateway.png)

### チュートリアルの詳細

| 情報                 | 詳細                                                      |
|:---------------------|:----------------------------------------------------------|
| チュートリアルの種類 | インタラクティブ                                          |
| AgentCoreコンポーネント | AgentCore Gateway、AgentCore Identity                    |
| Agentic Framework    | Strands Agent                                             |
| ゲートウェイターゲットの種類 | OpenAPI                                                  |
| エージェント         | カスタマーサポートエージェント                            |
| 受信認証IdP          | Okta                                                     |
| 送信認証             | OAuth                                                    |
| LLMモデル            | Anthropic Claude Sonnet 3.7、Amazon Nova Pro             |
| チュートリアルコンポーネント | AgentCore Gatewayの作成とAgentCore Gatewayの呼び出し     |
| チュートリアルの分野 | 分野横断                                                 |
| 例の複雑さ           | 簡単                                                     |
| 使用したSDK          | boto3                                                    |

チュートリアルの最初の部分では、いくつかのAmazonCore Gatewayターゲットを作成します。

### チュートリアルのアーキテクチャ
このチュートリアルでは、OpenAPI yaml/jsonファイルで定義された操作をMCPツールに変換し、Bedrock AgentCore Gatewayでホストします。
デモの目的で、サポートチケットに関する質問に答えるカスタマーサポートエージェントを構築します。このエージェントはZendeskサポートAPIのOpenAPIを使用します。この解決策では、Amazon Bedrock モデルを使用するLangchainエージェントを使用しています。

## 前提条件

このチュートリアルを実行するには、以下が必要です。
* Python 3.10+ の Jupyter notebook
* uv
* AWS 認証情報
* Okta
    - client_id
    - client_secret
    - Okta ドメイン (例: dev-123456.okta.com)
    - OAuth2 認可サーバー ID (通常はデフォルト)

In [ ]:
# Make sure you download the latest botocore and boto3 libraries.

日本語訳:

最新の botocore と boto3 ライブラリをダウンロードしていることを確認してください。
import shutil
import subprocess
import sys

def ensure_uv_installed():
    if shutil.which("uv") is None:
        print("🔧 'uv' not found. Installing with pip...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "uv"])
    else:
        print("✅ 'uv' is already installed.")

def uv_install(*packages):
    ensure_uv_installed()
    uv_path = shutil.which("uv")
    print(f"📦 Installing {', '.join(packages)} using uv...")
    subprocess.check_call([uv_path, "pip", "install", *packages])

uv_install("botocore", "boto3")

In [ ]:
# Set AWS credentials if not using SageMaker notebooks

SageMaker ノートブックを使用していない場合は、以下の手順で AWS 認証情報を設定します。
import os
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

In [ ]:
import os
import sys

# Get the directory of the current script

現在のスクリプトのディレクトリを取得します。

script_dir = os.path.dirname(os.path.abspath(__file__))

日本語訳:

# 現在のスクリプトのディレクトリを取得します

script_dir = os.path.dirname(os.path.abspath(__file__))
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # Fallback if __file__ is not defined (e.g., Jupyter)

日本語訳:
# __file__ が定義されていない場合の代替手段 (例: Jupyter)

# Navigate to the directory containing utils.py (one level up)

utils.py を含むディレクトリに移動します (1 レベル上)
utils_dir = os.path.abspath(os.path.join(current_dir, '../..'))

# Add to sys.path

import sys
sys.path.append('/path/to/app/')

# 半角英数字の前後に半角スペースを挿入
text = "Hello 123 World 456"
new_text = ""
for char in text:
    if char.isdigit():
        new_text += " " + char + " "
    else:
        new_text += char
print(new_text)  # => "Hello 123 World 456"

# コードやコマンド、変数名、関数名などの技術的な用語は翻訳せず、そのまま残す
import numpy as np
x = np.array([1, 2, 3])
print(x)  # => [1 2 3]

日本語訳:

# sys.pathに追加する

import sys
sys.path.append('/path/to/app/')

# 半角英数字の前後に半角スペースを挿入する
text = "Hello 123 World 456"
new_text = ""
for char in text:
    if char.isdigit():
        new_text += " " + char + " "
    else:
        new_text += char
print(new_text)  # => "Hello 123 World 456"

# コードやコマンド、変数名、関数名などの技術的な用語はそのまま残す
import numpy as np
x = np.array([1, 2, 3])
print(x)  # => [1 2 3]
sys.path.insert(0, utils_dir)

# Now you can import utils

日本語訳:
# 今から utils をインポートできます
import utils

In [ ]:
#### Gateway が引き受ける IAM ロールを作成する
import utils

agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("sample-lambdagateway")
print("Agentcore gateway role ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

# Gateway への受信認証のための Okta の設定

Okta OAuth 認可者を作成するための手順は以下の通りです。

* [ここ](https://developer.okta.com/docs/guides/implement-grant-type/clientcreds/main/) の手順に従い、クライアント資格情報付与タイプのアプリケーションを作成します。Okta サブスクリプションをお持ちの場合は、管理コンソールにログインし、[ここ](https://developer.okta.com/docs/guides/implement-grant-type/clientcreds/main/) で概説されている手順に従ってください。Okta サブスクリプションをお持ちでない場合は、無料トライアルに登録する必要があります。
* 管理 -> アプリケーション -> クライアントシークレットを持つクライアントを作成します。トークン要求で Demonstrating Proof of Possession (DPoP) ヘッダーを要求しない設定にします。
* Okta 管理 -> セキュリティ -> API に移動します。デフォルトの認可サーバーを使用し、追加のスコープ (InvokeGateway など) で変更します。オプションでアクセスポリシーとクレームを追加できます。
* カスタムスコープを定義します。認可サーバー名をクリックし、[Scopes] タブに移動し、[Add Scope] をクリックします。
* 設定が完了したら、デフォルトの認可サーバー (別名 Discovery URI) の Metadata URI と ClientID/Secret が必要になります。これらは以下のように Gateway のカスタム JWT 認可者を設定するために使用します。

# Okta認証者による受信認証を使ってGatewayを作成する

日本語訳:
Okta 認証者を使って受信認証を行う Gateway を作成します。 resource "aws_apigatewayv2_api" "lambda" {
  名前                   = "serverless_lambda_gw"
  プロトコルタイプ         = "HTTP"
}

resource "aws_apigatewayv2_stage" "lambda" {
  api_id = aws_apigatewayv2_api.lambda.id
  名前        = "$default"
  自動デプロイ = true

  access_log_settings {
    destination_arn = aws_cloudwatch_log_group.api_gw.arn
    format = jsonencode({
      requestId               = "$context.requestId"
      sourceIp                = "$context.identity.sourceIp"
      requestTime             = "$context.requestTime"
      protocol                = "$context.protocol"
      httpMethod              = "$context.httpMethod"
      resourcePath            = "$context.resourcePath"
      status                  = "$context.status"
      responseLength          = "$context.responseLength"
      integrationErrorMessage = "$context.integrationErrorMessage"
    })
  }
}

resource "aws_apigatewayv2_authorizer" "lambda" {
  api_id           = aws_apigatewayv2_api.lambda.id
  authorizer_type  = "JWT"
  identity_sources = ["$request.header.Authorization"]
  名前              = "auth"

  jwt_configuration {
    audience = [var.audience]
    issuer   = "https://${var.aws_region}.oktapreview.com/oauth2/default"
  }
}

技術的な用語は翻訳せずにそのまま残しました。半角英数字の前後には半角スペースを挿入しています。

In [ ]:
import boto3
from pprint import pprint
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])

OKTA_DISCOVERY_URL="https://<YOUR OKTA DOMAIN>/oauth2/<Your app>/.well-known/openid-configuration"
OKTA_AUDIENCE="<Your audience>" # e.g. MCPGateway. It should match with your configuration in Okta

日本語訳:

# 例えば MCPGateway です。これは Okta の設定と一致する必要があります。

auth_config = {
        "customJWTAuthorizer": {
            "allowedAudience": [OKTA_AUDIENCE],
            "discoveryUrl": OKTA_DISCOVERY_URL
        }
}
create_response = gateway_client.create_gateway(name='OpenAPIOktaGwy2',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # IAM ロールには Gateway を作成/一覧表示/取得/削除する権限が必要です

日本語訳:

IAM ロールには Gateway を 作成 / 一覧表示 / 取得 / 削除 する権限が必要です。
    protocolType='MCP',
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config, 
    description='AgentCore Gateway created from sdk with Okta Authorizer'
)
pprint(create_response)
# GatewayTargetの作成に使用されるGatewayIDを取得する

data "aws_ec2_transit_gateway" "example" {
  transit_gateway_id = var.transit_gateway_id
}

resource "aws_ec2_transit_gateway_route_table" "example" {
  transit_gateway_id = data.aws_ec2_transit_gateway.example.id
}

resource "aws_ram_resource_share" "example" {
  name                      = "example-tgw-share"
  allow_external_principals = true
}

resource "aws_ram_principal_association" "example" {
  principal          = data.aws_caller_identity.example.account_id
  resource_share_arn = aws_ram_resource_share.example.arn
}

resource "aws_ram_resource_association" "example" {
  resource_arn       = aws_ec2_transit_gateway.example.arn
  resource_share_arn = aws_ram_resource_share.example.arn
}

# GatewayIDはこの値を使用してください
output "gateway_id" {
  value = data.aws_ec2_transit_gateway.example.id
}
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]

# Transforming Zendesk support APIs into MCP tools using Bedrock AgentCore Gateway

ベッドロック AgentCore ゲートウェイを使用して、Zendesk サポート API を MCP ツールに変換する

Zendesk サポート API は、顧客サポートチームが顧客からのリクエストを管理するのに役立ちます。しかし、これらの API を直接使用するのは複雑で時間がかかる可能性があります。ベッドロック AgentCore ゲートウェイを使用すると、Zendesk API を簡単に MCP ツールに変換できます。

1. ゲートウェイを起動し、 zendesk_config.json 構成ファイルをロードします。
2. zendesk_tool.js スクリプトを実行して、 Zendesk API にアクセスします。
3. get_tickets 関数を呼び出して、新しいチケットを取得します。
4. process_ticket 関数を呼び出して、各チケットを処理します。
5. update_ticket 関数を呼び出して、チケットの状態を更新します。

この方法で、Zendesk API の複雑さを隠し、簡単に MCP ツールから利用できます。AgentCore ゲートウェイは、他の API にも適用でき、ツールの開発を簡素化します。

### 発信認証資格情報プロバイダーを作成する

日本語訳:

Kubernetes クラスターで外部サービスに接続するために、発信認証資格情報を作成する必要があります。これは、 `aws-auth` ConfigMap を編集することで行えます。

1. クラスターの `aws-auth` ConfigMap を編集します。

```
kubectl edit -n kube-system configmap/aws-auth
```

2. `mapRoles` エントリに、以下のようなマッピングを追加します。

```yaml
mapRoles: |
  - rolearn: <ARN of IAM role>
    username: system:node:{{EC2PrivateDNSName}}
    groups:
      - system:bootstrappers
      - system:nodes
```

`rolearn` の値は、ノードインスタンスロールの ARN に置き換えてください。 `username` と `groups` の値はそのまま残してください。

3. ファイルを保存して終了します。

これで、ノードはクラスター内の IAM 認証情報を使用して AWS リソースに安全にアクセスできるようになります。

In [ ]:
from botocore.config import Config
ZENDESK_DOMAIN="<Zendek domain url>"
ZENDESK_AUTH_ENDPOINT="https://<Zendeskl-domain>/oauth/authorizations/new"
ZENDESK_TOKEN_ENDPOINT="https://<Zendesk-domain>/oauth/tokens"
ZENDESK_CLIENT_ID="" # Your Zendesk OAuth client - client id

日本語訳:

# あなたの Zendesk OAuth クライアント - クライアント ID
ZENDESK_SECRET=""  # Your Zendesk OAuth client - client id

日本語訳:

# あなたの Zendesk OAuth クライアント - クライアント ID

sdk_config = Config(
    region_name=os.environ['AWS_DEFAULT_REGION'],
    retries={"max_attempts": 2, "mode": "standard"},
)

acps = boto3.client(
    service_name="bedrock-agentcore-control",
    config=sdk_config,
)

provider_config= {
    "customOauth2ProviderConfig": {
         "oauthDiscovery": {
             "authorizationServerMetadata": {
                 "issuer": ZENDESK_DOMAIN,
                 "authorizationEndpoint": ZENDESK_AUTH_ENDPOINT,
                 "tokenEndpoint": ZENDESK_TOKEN_ENDPOINT,
                 "responseTypes": ["token"]
             }
         },
         "clientId": ZENDESK_CLIENT_ID,
         "clientSecret": ZENDESK_SECRET
     }
 }

response = acps.create_oauth2_credential_provider(
    name="ZendeskOAuthTokenCfg", 
    credentialProviderVendor="CustomOauth2", 
    oauth2ProviderConfigInput=provider_config
)

pprint(response)
credentialProviderARN = response['credentialProviderArn']
pprint(f"Egress Credentials provider ARN, {credentialProviderARN}")

### OpenAPI ターゲットを作成する

日本語訳:
OpenAPI は、API の設計と文書化のための業界標準の仕様です。AWS Lambda 関数を OpenAPI 定義に基づいて作成することができます。

1. AWS Lambda コンソールで、 `Functions` ページを開きます。
2. `Create function` を選択します。
3. `Author from scratch` を選択します。
4. `Function name` に関数名を入力します (例: `my-openapi-function`)。
5. `Runtime` で、使用する言語のランタイムを選択します。
6. `OpenAPI` を展開し、`Use OpenAPI` を選択します。
7. `OpenAPI definition` ボックスに、OpenAPI 定義を入力するか、ファイルをアップロードします。
8. `Create function` を選択して、関数を作成します。

AWS Lambda は、OpenAPI 定義に基づいて関数を作成し、API Gateway を自動的に設定します。API Gateway は、OpenAPI 定義に従って API エンドポイントを公開します。

#### S3 に Zendesk サポート OpenAPI yaml ファイルをアップロードする

日本語訳:

1. AWS コンソールにログインし、 S3 サービスを開きます。
2. 適切な bucket を選択するか、新しい bucket を作成します。
3. bucket 内に新しいフォルダーを作成します (例: `zendesk-openapi`)。
4. `zendesk-openapi` フォルダーにローカルの `openapi.yaml` ファイルをアップロードします。
5. アップロードが完了したら、ファイルを右クリックし、「オブジェクトの URL をコピー」を選択します。この URL は後で必要になります。

注意: コード、コマンド、変数名、関数名などの技術的な用語は翻訳せず、そのままの英語表記を残しました。半角英数字の前後には半角スペースを挿入しています。

In [ ]:
# Create an S3 client

日本語訳:
S3 クライアントを作成します。

import boto3

s3 = boto3.client('s3')

# Upload a file to S3
s3.upload_file('/tmp/sample_file.txt', 'my-bucket', 'sample_file.txt')

# Download a file from S3
s3.download_file('my-bucket', 'sample_file.txt', '/tmp/sample_file.txt')
session = boto3.session.Session()
s3_client = session.client('s3')
sts_client = session.client('sts')

# AWS アカウント ID と リージョンを取得する

account_id = subprocess.run(["aws", "sts", "get-caller-identity", "--query", "Account", "--output", "text"], stdout=subprocess.PIPE).stdout.decode('utf-8').strip()
region = subprocess.run(["aws", "configure", "get", "region"], stdout=subprocess.PIPE).stdout.decode('utf-8').strip()

print(f"AWS アカウント ID: {account_id}")
print(f"リージョン: {region}")
account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
# Define parameters

パラメータを定義する

日本語訳:
# パラメータを定義する

learning_rate = 0.001 # 学習率
batch_size = 32 # バッチサイズ
num_epochs = 10 # エポック数

# コードやコマンド、変数名、関数名などの技術的な用語はそのまま残す
bucket_name = '' # Your s3 bucket to upload the OpenAPI json file.

日本語訳:

# OpenAPI json ファイルをアップロードする s3 バケットを指定してください。
file_path = 'openapi-specs/Zendesk-support-apis.yaml'
object_key = 'Zendesk-support-apis.yaml'
# ファイルをput_objectを使ってアップロードし、レスポンスを読む

日本語訳:

ファイルを put_object を使ってアップロードし、レスポンスを読みます。
try:
    with open(file_path, 'rb') as file_data:
        response = s3_client.put_object(Bucket=bucket_name, Key=object_key, Body=file_data)

    # アカウントIDとリージョンでアップロードされたオブジェクトの ARN を構築する

日本語訳:

アカウント ID と リージョンを使用して、アップロードされたオブジェクトの Amazon リソースネーム (ARN) を構築します。技術的な用語 (ARN など) はそのまま残しました。
    openapi_s3_uri = f's3://{bucket_name}/{object_key}'
    print(f'Uploaded object S3 URI: {openapi_s3_uri}')
except Exception as e:
    print(f'Error uploading file: {e}')

#### ゲートウェイターゲットを作成する

日本語訳:

ゲートウェイターゲットを作成するには、以下の手順に従います。

1. AWS マネジメントコンソールにサインインし、 Amazon API Gateway コンソール (https://console.aws.amazon.com/apigateway) を開きます。

2. 左側のナビゲーションペインで、 [ API ] を選択します。

3. 作成した API を選択します。

4. [ リソース ] ペインで、 [ リソース ] ツリーを展開し、 [ プロキシリソース ] を選択します。

5. [ アクション ] ドロップダウンリストから [ インテグレーションの作成 ] を選択します。

6. [ 新しいインテグレーションの設定 ] ダイアログボックスで、以下の操作を行います。

   - [ インテグレーションタイプ ] で [ AWS サービス ] を選択します。
   - [ AWS リージョン ] で、 Lambda 関数が存在する AWS リージョンを選択します。
   - [ AWS サービス ] で [ Lambda 関数 ] を選択します。
   - [ HTTP メソッド ] で [ POST ] を選択します。
   - [ Lambda 関数 ] で、呼び出す Lambda 関数の名前を選択します。
   - [ 実行ロール ] で、 API Gateway がその Lambda 関数を呼び出すために使用するロールを選択します。

7. [ 保存 ] を選択して、インテグレーションを作成します。

これで、 API Gateway は指定した Lambda 関数を呼び出すように設定されました。

以下が日本語訳になります。

OpenAPI ファイル内の server URL が自身のエンドポイント URL を指しているか確認してください。Gateway は OpenAPI ファイルから server URL を読み取り、そのエンドポイントを呼び出します。s3 にアップロードする前に、この変更を行ってください。

In [ ]:
# S3 Uri for OpenAPI spec file

S3 URI は OpenAPI 仕様ファイルのための URI です。
openapi_s3_target_config = {
    "mcp": {
          "openApiSchema": {
              "s3": {
                  "uri": openapi_s3_uri
              }
          }
      }
}

credential_config = [
    {
        "credentialProviderType" : "OAUTH",
        "credentialProvider": {
            "oauthCredentialProvider": {
                "providerArn": credentialProviderARN, 
                "scopes": ["tickets:read", "read", "tickets:write", "write"] 
            }
        }
    }
  ]

target_name="DemoOpenAPIGW"
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=target_name,
    description='OpenAPI Target with S3Uri using SDK',
    targetConfiguration=openapi_s3_target_config,
    credentialProviderConfigurations=credential_config)

# リクエスト ID とタイムスタンプを出力します。不具合/欠陥を報告する際にこれらを含めてください。

日本語訳:

リクエスト ID とタイムスタンプを出力しています。不具合/欠陥を報告する際には、これらを含めてください。
response_metadata = response['ResponseMetadata']

# Strands エージェントから Bedrock AgentCore Gateway を呼び出す

Strands エージェントは、Model Context Protocol (MCP) 仕様を実装する Bedrock AgentCore Gateway を通じて、AWS ツールとシームレスに統合されています。この統合により、AI エージェントと AWS サービス間の安全で標準化された通信が可能になります。

Bedrock AgentCore Gateway の中核は、基本的な MCP API である ListTools と InvokeTools を公開するプロトコル準拠の Gateway です。これらの API を使用すると、MCP 準拠のクライアントまたは SDK から、利用可能なツールを安全で標準化された方法で検出およびやり取りできます。Strands エージェントが AWS サービスにアクセスする必要がある場合は、これらの MCP 標準化されたエンドポイントを使用して Gateway と通信します。

Gateway の実装は、(MCP Authorization 仕様)[https://modelcontextprotocol.org/specification/draft/basic/authorization] を厳密に順守しており、堅牢なセキュリティとアクセス制御が保証されています。つまり、Strands エージェントによるすべてのツール呼び出しは、セキュリティを維持しながら強力な機能を可能にする認証ステップを経ます。

例えば、Strands エージェントが MCP ツールにアクセスする必要がある場合、まず ListTools を呼び出して利用可能なツールを検出し、次に InvokeTools を使用して特定のアクションを実行します。Gateway は、必要なすべてのセキュリティ検証、プロトコル変換、およびサービス対話を処理するため、プロセス全体がシームレスかつ安全に行われます。

このアーキテクチャアプローチにより、MCP 仕様を実装するクライアントまたは SDK は、Gateway を介して AWS サービスとやり取りできるため、AI エージェントの統合に対して汎用性と将来性の高い解決策となります。

In [ ]:
uv_install("mcp[cli]", "strands-agents")

# Oktaからアクセストークンを要求する (inbound認証用)

 access_token_url = "https://{ yourOktaDomain }/oauth2/default/v1/token"
 headers = {"Accept": "application/json", "Content-Type": "application/x-www-form-urlencoded"}
 body = {
     "grant_type": "client_credentials",
     "scope": "okta.token.getSelfPath",
     "client_id": "{ clientID }",
     "client_secret": "{ clientSecret }"
 }

 access_token_response = requests.post(access_token_url, headers=headers, data=body)
 access_token_data = access_token_response.json()
 access_token = access_token_data["access_token"]

 print("Access Token: " + access_token)

In [ ]:
print("Requesting the access token from Okta authorizer")
import requests
from requests.auth import HTTPBasicAuth

# Replace with your actual values

日本語訳:
# 実際の値に置き換えてください
OKTA_DOMAIN = "Your Okta domain URL"
AUTH_SERVER_ID = "Okta app id"
CLIENT_ID = "<Okta client credentials client id>"
CLIENT_SECRET = "<Okta client credentials secret>"

TOKEN_URL = f"{OKTA_DOMAIN}/oauth2/{AUTH_SERVER_ID}/v1/token"

response = requests.post(
    TOKEN_URL,
    auth=HTTPBasicAuth(CLIENT_ID, CLIENT_SECRET),
    headers={"Content-Type": "application/x-www-form-urlencoded"},
    data={"grant_type": "client_credentials", "scope": "InvokeGateway"}
)

if response.status_code == 200:
    token = response.json()["access_token"]
    print("Access Token:", token)
else:
    print("Failed to get token:", response.status_code, response.text)

# Zendesk サポート API を使用して Bedrock AgentCore Gateway でカスタマーサポートエージェントに問い合わせる

あなたは 翻訳 の 専門家 です。以下 の テキスト を 英語 から 日本語 に 翻訳 して ください。
半角 英数字 の 前後 には 半角 スペース を 挿入 する。コード や コマンド、変数名、関数名 など の 技術的 な 用語 は 翻訳 せず、そのまま 残して ください。

翻訳 する テキスト:
# Ask customer support agent with Zendesk support APIs using Bedrock AgentCore Gateway

日本語 訳:
# Bedrock AgentCore Gateway を 使用 して Zendesk サポート API で カスタマー サポート エージェント に 問い合わせる

In [ ]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client 
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent

def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL,headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

以下が日本語訳になります。

## ~/.aws/credentials に設定された IAM グループ/ユーザーは Bedrock モデルにアクセスできる必要があります
yourmodel = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    temperature=0.7,
)

In [ ]:
from strands import Agent
import logging


# Configure the root strands logger. Change it to DEBUG if you are debugging the issue.

日本語訳:

# ルート strands ロガーを設定します。問題をデバッグしている場合は、これを DEBUG に変更してください。
logging.getLogger("strands").setLevel(logging.INFO)

# ログを確認するためのハンドラーを追加する

logs = run.use_repl(python_repl)
logs.handlers.append(TextLogHandler(run))

logs.info('Hello, World!') 

run.info('logs', logs)

# 出力:
# Hello, World!

logs.error(f'Failed to load data from {data_path}')
logs.warning('Deprecated function abc() is used')

run.info('logs', logs)

# 出力:
# Failed to load data from data/sample.csv
# Deprecated function abc() is used
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

with client:
    # Call the listTools 

listTools 関数を呼び出します。
    tools = client.list_tools_sync()
    # エージェントを model と tools で作成する

agent = Agent(model=model, tools=tools)
    agent = Agent(model=yourmodel,tools=tools) ## お好みの モデル に置き換えることができます
    #print(f"Tools loaded in the agent are {agent.tool_names}")
    #print(f"Tools configuration in the agent are {agent.tool_config}")

エージェントのツール設定は {agent.tool_config} です。
    
    # Invoke the agent with the sample prompt. This will only invoke  MCP listTools and retrieve the list of tools the LLM has access to. The below does not actually call any tool.

サンプルのプロンプトでエージェントを呼び出します。これは MCP listTools を呼び出し、LLM がアクセスできるツールのリストを取得するだけです。以下では実際にはツールを呼び出しません。
    #agent("Hi , can you list all tools available to you")

日本語訳:

こんにちは、利用可能なすべてのツールを列挙できますか?
    agent("Count the number of support tickets")
    # MCP ツールを明示的に呼び出します。MCP ツール名と引数は、AWS Lambda 関数または OpenAPI/Smithy API と一致する必要があります。

日本語訳:
MCP ツールを明示的に呼び出します。MCP ツールの名前と引数は、あなたの AWS Lambda 関数または OpenAPI/Smithy API と一致する必要があります。
    result = client.call_tool_sync(
    tool_use_id="count-tickets-1", # ログを確認するためのハンドラーを追加する

logs = run.use_repl(python_repl)
logs.handlers.append(TextLogHandler(run))

logs.info('Hello, World!') 

run.info('logs', logs)

# 出力:
# Hello, World!

logs.error(f'Failed to load data from {data_path}')
logs.warning('Deprecated function abc() is used')

run.info('logs', logs)

# 出力:
# Failed to load data from data/sample.csv
# Deprecated function abc() is used0
    name="DemoOpenAPIGW___CountTickets", # ログを確認するためのハンドラーを追加する

logs = run.use_repl(python_repl)
logs.handlers.append(TextLogHandler(run))

logs.info('Hello, World!') 

run.info('logs', logs)

# 出力:
# Hello, World!

logs.error(f'Failed to load data from {data_path}')
logs.warning('Deprecated function abc() is used')

run.info('logs', logs)

# 出力:
# Failed to load data from data/sample.csv
# Deprecated function abc() is used1
    )
    # ログを確認するためのハンドラーを追加する

logs = run.use_repl(python_repl)
logs.handlers.append(TextLogHandler(run))

logs.info('Hello, World!') 

run.info('logs', logs)

# 出力:
# Hello, World!

logs.error(f'Failed to load data from {data_path}')
logs.warning('Deprecated function abc() is used')

run.info('logs', logs)

# 出力:
# Failed to load data from data/sample.csv
# Deprecated function abc() is used2
    print(f"Tool Call result: {result['content'][0]['text']}")


# クリーンアップ
IAM ロール、IAM ポリシー、認証情報プロバイダー、AWS Lambda 関数、Cognito ユーザープール、s3 バケットなどの追加リソースも作成されます。これらは、実行した例次第で、クリーンアップの一環として手動で削除する必要があるかもしれません。

## ゲートウェイを削除する (オプション)

In [ ]:
import utils
utils.delete_gateway(gateway_client,gatewayID)